In [ ]:
#MS data pre-analysis by MS-DIAL 5.3. Only peak table as txt. format was needed.

import pandas as pd
import numpy as np
import re
import os
import glob
import time

minint = 0.01
ppm = 1e-6  # Global variable.

NLmin = 2         # The minimum Da of NL retained in the results
Round_NL = 2       # Keep the number of decimal digits NL is calculated
Round_EG = 1

Mode = "negative"

# Function to get the adduct mass based on the mode (positive/negative) and adduct type
def Get_Adduct_Mass(Mode, Adduct):
    Z = 0
    
    if Mode == "negative":
        if Adduct == "[M-H]-":
            Z = -1.0072766
        elif Adduct == "[M+H2O-H]-":
            Z = 17.0032881
        elif Adduct == "[M+Cl]-":
            Z = 33.9615761
        elif Adduct == "[M+FA-H]-":
            Z = 44.9982027
        elif Adduct == "[M+Hac-H]-":
            Z = 59.0138527
        elif Adduct == "[M-2H]2-":
            Z = -2.0145532
    
    if Mode == "positive":
        if Adduct == "[M+H]+":
            Z = 1.0072767
        elif Adduct == "[M+Na]+":
            Z = 22.9892213
        elif Adduct == "[M+K]+":
            Z = 38.9631585
        elif Adduct == "[M+H-H2O]+":
            Z = -17.0021912
        elif Adduct == "[M+ACN+H]+":
            Z = 42.0338258
        elif Adduct == "[M+CH3OH+H]+":
            Z = 33.0334914
        elif Adduct == "[M+NH4]+":
            Z = 19.0416508
        elif Adduct == "[M+2H]2+":
            Z = 2.0145533
    
    return Z

# Get all the txt files in the current directory
txt_files = glob.glob("*.txt")    #The input raw file is a txt peak table exported from MSDIAL 5.3.

start_time = time.time()

for txt_file in txt_files:
    file_start_time = time.time()
    
    # Read MS1 peak table
    peak_table = pd.read_table(txt_file)
    output_path = txt_file.replace('.txt', '.xlsx')

    # Add a new column 'Mass' to the peak_table
    peak_table['Mass'] = 0.0

    # Iterate through each row of the DataFrame
    for index, row in peak_table.iterrows():
        Adduct = row['Adduct']
        Charge = re.sub(".*\\](.)", "\\1", Adduct)
        Charge = Charge[:1]

        if Charge == "+" or Charge == "-":
            Charge = 1.0  # Convert to float
        else:
            Charge = float(Charge)  # Convert to float
        
        Precursor = row['Precursor m/z']
        Z = Get_Adduct_Mass(Mode, Adduct)
        
        Mass = Precursor * Charge - Z
        peak_table.at[index, 'Mass'] = Mass
        
    # Add new columns 'rNL', 'Count', 'Frequency', 'DP', and 'EG' to the peak_table
    peak_table['rNL'] = 0.0
    peak_table['Count'] = 0.0
    peak_table['Frequency'] = 0.0
    peak_table['DP'] = 0.0
    peak_table['EG'] = 0.0

    # Iterate through all peaks
    for i in range(len(peak_table)):
        Mass = peak_table["Mass"][i]
        Precursor = peak_table["Precursor m/z"][i]
        ms2peak_table = str(peak_table["MSMS spectrum"][i]).split(";")  # Split by semicolon
        if ms2peak_table != ['nan']:
            # Clean spectra
            ms2peak_table = pd.DataFrame([x.split(" ") for x in ms2peak_table], columns=["mz", "intensity"])
            ms2peak_table['intensity'] = pd.to_numeric(ms2peak_table['intensity'])
            ms2peak_table['mz'] = pd.to_numeric(ms2peak_table['mz'])
            # Calculate maxprecursor_differ and maxprecursor
            maxprecursor_differ = 1 + 10 * ppm
            maxprecursor = Precursor * maxprecursor_differ
            # Filter rows where mz is less than or equal to maxprecursor
            ms2peak_table = ms2peak_table[ms2peak_table['mz'] <= maxprecursor]
            ms2peak_table["Rintensity"] = ms2peak_table['intensity'] / max(ms2peak_table['intensity'])
            # Filter rows where relative intensity is not less than minint
            ms2peak_table = ms2peak_table[ms2peak_table['Rintensity'] >= minint]
            # Reset index
            ms2peak_table.reset_index(drop=True, inplace=True)
        
            # Calculate NL candidates
            mz = ms2peak_table['mz'].to_numpy()
            N = len(mz)
            b = []
            if N == 1:
                b.append(0)
            else:
                for k in range(N):
                    for j in range(k + 1, N):
                        a = abs(mz[k] - mz[j])
                        b.append(a)
            # Round to the specified number of decimal places
            NL = np.round(b, Round_NL)
            NL = NL[NL >= NLmin]
            
            # Create frequency table
            d = pd.DataFrame(NL, columns=['rNL']).value_counts().reset_index(name='Count')
            
            # Calculate frequency percentage
            d['Freq_Percentage'] = d['Count'] / len(NL)
            
            # Keep only the most frequent NL, i.e., rNL
            max_freq_count = d['Count'].max()
            e = d[d['Count'] == max_freq_count].copy()
            # Sort by rNL column in descending order
            sorted_df = e.sort_values(by='rNL', ascending=False)

            # Reorder index
            e = sorted_df.reset_index(drop=True)
            if not e.empty:
                peak_table.at[i, 'rNL'] = e.iloc[0, 0]
                peak_table.at[i, 'Count'] = e.iloc[0, 1]
                peak_table.at[i, 'Frequency'] = e.iloc[0, 2]
                peak_table.at[i, 'DP'] = Mass // peak_table.at[i, 'rNL']
                peak_table.at[i, 'EG'] = Mass % peak_table.at[i, 'rNL']
                peak_table.at[i, 'EG'] = peak_table.at[i, 'EG'].round(Round_EG)
                
    # Delete columns 14 to 28
    peak_table = peak_table.drop(peak_table.columns[10:37], axis=1) 
    
    # Read PMDB
    peak_table['RU'] = 'Unknown'
    peak_table['rNLmass'] = 'Unknown'
    PMDB = pd.read_excel('Polymer oligomer mass database.xlsx')
    for i in range(len(peak_table)):
        for j in range(len(PMDB)):
            if peak_table.at[i, 'rNL'] == PMDB.at[j, 'NL']:
                peak_table.at[i, 'rNLmass'] = PMDB.at[j, 'NL mass']
                peak_table.at[i, 'RU'] = PMDB.at[j, 'Names']
                break
                
    # Read EGDB
    peak_table['EG.formula'] = 'Unknown'
    peak_table['EG.mass'] = 'Unknown'
    EGDB = pd.read_excel('End group database.xlsx')
    for i in range(len(peak_table)):
        for j in range(len(EGDB)):
            if peak_table.at[i, 'EG'] == EGDB.at[j, 'EG']:
                peak_table.at[i, 'EG.mass'] = EGDB.at[j, 'EG.mass']
                peak_table.at[i, 'EG.formula'] = EGDB.at[j, 'EG.formula']
                break
                
    # Filter rows where 'Count' >= 2 and 'DP' >= 3
    peak_table = peak_table[(peak_table['Count'] >= 2) & (peak_table['DP'] >= 3)]           
    # Export the processed DataFrame to an Excel file
    peak_table.to_excel(output_path, index=False)

    file_end_time = time.time()
    print(f"Processed {txt_file} and saved as {output_path}. Time taken: {file_end_time - file_start_time:.2f} seconds.")

end_time = time.time()
print(f"All files have been processed. Total time taken: {end_time - start_time:.2f} seconds.")
